In [1]:
import pandas as pd
import os
from mlflow.tracking import MlflowClient


from utils import get_params_and_metrics
from utils import METRICS, HYPERPARAMETERS, ARCHITECTURE, DATASETS, OTHERS

NOTEBOOK_URI="sqlite:////home/guillermo/Documents/SINC/Research/RNA/AEseq2seq/mlruns.db"
SINC_URI="sqlite:////home/gkulemeyer/Documents/Repos/seq2seq/results/mlflow/mlruns.db"

# Conectar al backend store
client = MlflowClient(tracking_uri=SINC_URI)
experiments = client.search_experiments()
for i,exp in enumerate(experiments):
    print(f"i: {i}, ID: {exp.experiment_id}, Nombre: {exp.name}, Estado: {exp.lifecycle_stage}")

i: 0, ID: 4, Nombre: Train_test_w_masking, Estado: active
i: 1, ID: 3, Nombre: Test_noise_w_noise, Estado: active
i: 2, ID: 2, Nombre: Train_w_noise_0_1, Estado: active
i: 3, ID: 1, Nombre: Test_script, Estado: active
i: 4, ID: 0, Nombre: Default, Estado: active


In [10]:
pd.options.display.max_columns = 999

In [6]:
n = 0
name = experiments[n].name
id = experiments[n].experiment_id
print(f"Nombre: {name}, ID: {id}, Estado: {experiments[n].lifecycle_stage}")

df_arc = get_params_and_metrics(client, id, ARCHITECTURE  + HYPERPARAMETERS)
for col in ARCHITECTURE:
    print(f"{col}: {df_arc[col].unique()}")
for col in HYPERPARAMETERS:
    print(f"{col}: {df_arc[col].unique()}")

Nombre: Train_test_w_masking, ID: 4, Estado: active
arc_embedding_dim: ['4']
arc_features: ['[4, 4, 4, 8]' '[4, 4, 8]' '[4, 8]' '[4, 4, 4]' '[4, 4]' '[4]']
arc_encoder_blocks: ['3' '2' '1' '0']
arc_num_params: ['692' '548' '524' '428' '356' '308' '416' '320' '248' '200' '80']
arc_initial_volume: ['512']
arc_latent_volume: ['128.0' '256.0' '512.0']
arc_num_conv: ['1']
arc_pool_mode: ['max']
arc_up_mode: ['upsample']
arc_addition: ['cat']
arc_skip: ['1' '0']
hyp_device: ['cuda']
hyp_lr: ['0.001']
hyp_scheduler: ['none']
hyp_verbose: ['True']
max_epochs: ['30']
test_swaps: ['1' '0.9' '0.8' '0.7' '0.6' '0.5' '0.4' '0.3' '0.2' '0.1' '0.0' nan]


In [11]:
metr = ['test_loss', 
 'test_F1', 
 'test_Accuracy', 
 'test_Accuracy_seq']

In [19]:
n = 0
name = experiments[n].name
id = experiments[n].experiment_id
print(f"Nombre: {name}, ID: {id}, Estado: {experiments[n].lifecycle_stage}")

# METRICS, HYPERPARAMETERS, ARCHITECTURE, DATASETS, OTHERS
EXTRACT = [ 
    "arc_features",
    "arc_encoder_blocks",
    "arc_num_params",
    "arc_latent_volume",
    "arc_num_conv",
    "arc_skip", 
    "test_swaps",
]

# df_metrics = get_params_and_metrics(client, id, EXTRACT + METRICS)
df_metrics = get_params_and_metrics(client, id, EXTRACT + METRICS)
print(f"df shape: {df_metrics.shape}")
df_metrics.sample(6)

Nombre: Train_test_w_masking, ID: 4, Estado: active
df shape: (1584, 20)


,run_name,arc_features,arc_encoder_blocks,arc_num_params,arc_latent_volume,arc_num_conv,arc_skip,test_swaps,train_loss,valid_loss,test_loss,valid_F1,train_F1,test_F1,train_Accuracy,valid_Accuracy,test_Accuracy,train_Accuracy_seq,valid_Accuracy_seq,test_Accuracy_seq
761,skip0_noise_0.5_n4_1_n8_1-0.5,"[4, 4, 8]",2,428,256.0,1,0,0.5,NaN,NaN,0.135605,NaN,NaN,0.446433,NaN,NaN,0.446433,NaN,NaN,0.0
453,skip0_noise_0.7_n4_2_n8_1-0.1,"[4, 4, 4, 8]",3,548,128.0,1,0,0.1,NaN,NaN,0.144272,NaN,NaN,0.370142,NaN,NaN,0.370142,NaN,NaN,0.0
503,skip0_noise_0.7_n4_0_n8_1,"[4, 8]",1,308,512.0,1,0,NaN,0.129483,0.088820,NaN,0.730091,0.450856,NaN,0.450856,0.730091,NaN,0.0,0.0,NaN
1468,skip1_noise_0.0_n4_1_n8_1-0.6,"[4, 4, 8]",2,524,256.0,1,1,0.6,NaN,NaN,0.117712,NaN,NaN,0.562881,NaN,NaN,0.562881,NaN,NaN,0.0
652,skip1_noise_0.6_n4_2_n8_0-0.6,"[4, 4, 4]",2,416,128.0,1,1,0.6,NaN,NaN,0.089849,NaN,NaN,0.576769,NaN,NaN,0.576769,NaN,NaN,0.0
491,skip1_noise_0.7_n4_0_n8_1,"[4, 8]",1,356,512.0,1,1,NaN,0.108678,0.002949,NaN,1.000000,0.519651,NaN,0.519651,1.000000,NaN,0.0,0.0,NaN


In [31]:
df_metrics['name'] = df_metrics['run_name']
df_metrics['name']

0           skip1_noise_1_n4_2_n8_1-1
1         skip1_noise_1_n4_2_n8_1-0.9
2         skip1_noise_1_n4_2_n8_1-0.8
3         skip1_noise_1_n4_2_n8_1-0.7
4         skip1_noise_1_n4_2_n8_1-0.6
                    ...              
1579    skip0_noise_0.0_n4_0_n8_0-0.3
1580    skip0_noise_0.0_n4_0_n8_0-0.2
1581    skip0_noise_0.0_n4_0_n8_0-0.1
1582    skip0_noise_0.0_n4_0_n8_0-0.0
1583        skip0_noise_0.0_n4_0_n8_0
Name: name, Length: 1584, dtype: object

In [18]:
df_metrics['train_swaps'] = df_metrics['run_name'].str.extract(r'noise_([0-9.]+)')
df_metrics['train_swaps'] = df_metrics['train_swaps'].astype(float)
df_metrics

,run_name,arc_features,arc_encoder_blocks,arc_num_params,arc_latent_volume,arc_num_conv,arc_skip,test_swaps,train_loss,valid_loss,test_loss,valid_F1,train_F1,test_F1,train_Accuracy,valid_Accuracy,test_Accuracy,train_Accuracy_seq,valid_Accuracy_seq,test_Accuracy_seq,train_swaps
0,skip1_noise_1_n4_2_n8_1-1,"[4, 4, 4, 8]",3,692,128.0,1,1,1,NaN,NaN,1.842996e-01,NaN,NaN,0.280307,NaN,NaN,0.280307,NaN,NaN,0.0,1.0
1,skip1_noise_1_n4_2_n8_1-0.9,"[4, 4, 4, 8]",3,692,128.0,1,1,0.9,NaN,NaN,6.650820e+10,NaN,NaN,0.230927,NaN,NaN,0.230927,NaN,NaN,0.0,1.0
2,skip1_noise_1_n4_2_n8_1-0.8,"[4, 4, 4, 8]",3,692,128.0,1,1,0.8,NaN,NaN,8.353495e+10,NaN,NaN,0.234276,NaN,NaN,0.234276,NaN,NaN,0.0,1.0
3,skip1_noise_1_n4_2_n8_1-0.7,"[4, 4, 4, 8]",3,692,128.0,1,1,0.7,NaN,NaN,1.045507e+11,NaN,NaN,0.238491,NaN,NaN,0.238491,NaN,NaN,0.0,1.0
4,skip1_noise_1_n4_2_n8_1-0.6,"[4, 4, 4, 8]",3,692,128.0,1,1,0.6,NaN,NaN,1.295236e+11,NaN,NaN,0.239177,NaN,NaN,0.239177,NaN,NaN,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1579,skip0_noise_0.0_n4_0_n8_0-0.3,[4],0,80,512.0,1,0,0.3,NaN,NaN,5.727893e-02,NaN,NaN,0.783240,NaN,NaN,0.783240,NaN,NaN,0.0,0.0
1580,skip0_noise_0.0_n4_0_n8_0-0.2,[4],0,80,512.0,1,0,0.2,NaN,NaN,4.073744e-02,NaN,NaN,0.854851,NaN,NaN,0.854851,NaN,NaN,0.0,0.0
1581,skip0_noise_0.0_n4_0_n8_0-0.1,[4],0,80,512.0,1,0,0.1,NaN,NaN,2.436779e-02,NaN,NaN,0.926847,NaN,NaN,0.926847,NaN,NaN,0.0,0.0
1582,skip0_noise_0.0_n4_0_n8_0-0.0,[4],0,80,512.0,1,0,0.0,NaN,NaN,7.831934e-03,NaN,NaN,1.000000,NaN,NaN,1.000000,NaN,NaN,0.0,0.0


In [32]:
EXPERIMENT_PATH = os.path.join("Masking", name)
os.makedirs(EXPERIMENT_PATH, exist_ok=True)
df_metrics.to_csv(os.path.join(EXPERIMENT_PATH, "metrics.csv"), index=False)